# Using HELR for Private Logistic Regression Inference with Homomorphic Encryption

The code below shows a use case of the HELR library for making private inference with a Neural Network.

In this case, we are using the LogisticRegression notebook to create the logistic regression and we extract eht

In [1]:
import numpy as np
import tensorflow as tf
from utils import * # My module

2022-02-03 16:40:49.875597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-03 16:40:49.875637: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
!pip install ../src/dist/*.whl --force-reinstall
from HELR import HELR

Processing /home/jocabrer/mnt/src/dist/HELR-0.1.0-cp39-cp39-linux_x86_64.whl
  Attempting uninstall: HELR
    Found existing installation: HELR 0.1.0
    Uninstalling HELR-0.1.0:
      Successfully uninstalled HELR-0.1.0


### Loading the dataset & model

In [3]:
x_train, y_train, x_val, y_val, x_test, y_test = load_dataset("./dataset/")

In [4]:
model = load_model("model/lr.h5")

2022-02-03 16:40:57.763571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-03 16:40:57.763611: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-03 16:40:57.763635: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (he_mia): /proc/driver/nvidia/version does not exist
2022-02-03 16:40:57.763965: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Selecting a sample image

In [5]:
sample_img = x_train[1,:,:,0]

In [6]:
sample_img.shape

(128, 128)

### Using HELR to get the model params from a Tensorflow model
Note that: The `get_lr_params` is meant to work with Tensorflow

In [7]:
w, b = HELR.get_lr_params(model)

### Define the HELR object

It internally caches the weights (`w`) and bias values (`b`)

In [8]:
helr = HELR(w, b)

### Executing the private inference

The call to the predict function with a sample image generates a private prediction so that:
- It generates the keys for the processing.
- It encrypts the image to a ciphertext.
- It encodes the weights and bias in a plaintext.
- It uses those to perform the private linear phase of the neural network.
- It decrypts the values.
- It performs a cleartext sigmoid function.
- Returns the result.


In [9]:
helr.predict(sample_img)

6.617451721677384e-16

### Skipping the key generation

The key generation procedure takes most of the time and its randomness is in general of minimal effect to the computation.

For the sake of research and testing, the key generation can be omitted setting `reuse_keys=True`. 

Note that this should not be done on a real deployment.

In [10]:
helr.predict(sample_img, reuse_keys=True)

6.617469004505134e-16

In [11]:
%time helr.predict(sample_img, reuse_keys=False)
%time helr.predict(sample_img, reuse_keys=True)

CPU times: user 3.14 s, sys: 114 ms, total: 3.26 s
Wall time: 3.25 s
CPU times: user 600 ms, sys: 29.8 ms, total: 630 ms
Wall time: 631 ms


6.617015791062156e-16